In [8]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from plotly import graph_objects as go

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_pop_death_data, get_death_rates
from tb_incubator.model import build_model
from tb_incubator.input import load_param_info
from tb_incubator.plotting import display_plot

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

We used estimates for the total population, deaths, and birth rate from the United Nations World Population Prospects [@unwpp2024].

In [ ]:
# Load age-stratified, population and death data
pop_death, description = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]
Markdown(description)

## Model construction
This is arbitrary, but including some epidemiological transitions in the background.

In [3]:
compartments = const.compartments
age_strata = const.age_strata

In [ ]:
param_info = load_param_info()
params = param_info["value"]

model, desc = build_model(params, xpert_sensitivity=True, covid_effects=True)
Markdown(desc)

In [5]:
# age-stratified population
age_pops_output = [model.request_output_for_compartments(f"total_populationXage_{s}", compartments, strata={"age": str(s)}) for s in age_strata]


## Demographic results

In [ ]:
# Run and inspect results
file_prefix = "ID"
model.run(params)
fig = model.get_derived_outputs_df()[[f"total_populationXage_{age}" for age in age_strata]].plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))
#display_plot(fig, f"{file_prefix}_comp_size", "svg")